## スクレイピング賃貸編

- goenでのteam開発を意識して「賃貸」画面のスクレイピングに取り組む
- 一人暮らしの人が近隣の情報も検索するという今後の開発意図を考慮して、そのようなペルソナが使いそうな検索から物件情報を取得する
- 想像では、沿線検索を利用するのではないかと仮定して、日比谷線沿線の物件情報の取得を以下に書いていく → うまくいかない
- 地域情報からで検索することとする（荒川区）

In [2]:
#おまじない
#importする順番はアルファベット順が良い！
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep

#printを使うと見辛いのでpprintをインポートする
from pprint import pprint

In [3]:
#変数URLにSUUMOのホームページURLを格納する
url = 'https://suumo.jp/chintai/tokyo/sc_arakawa/?page={}'

#アクセスするためのURLをtarget_urlに格納する
target_url = url.format(1)

#print()してtarget_urlを確認する
print(target_url)

https://suumo.jp/chintai/tokyo/sc_arakawa/?page=1


In [4]:
#target_urlへのアクセス結果を変数rに格納する
r = requests.get(target_url)

#取得結果を解析してsoupに格納する
soup = BeautifulSoup(r.text,"html.parser")

In [5]:
#soup

In [6]:
print(r.encoding) # res.encodingで文字エンコードを確認できます。

UTF-8


In [7]:
#HTMLを確認→それぞれの賃貸情報はcassetteitemに格納されている
contents = soup.find_all('div',class_='cassetteitem')

In [8]:
#contents

In [9]:
#find_allで取得した結果はリスト形式になっているのでlenで要素数を確認。1pあたり２０件
len(contents)

20

In [10]:
content = contents[0]

In [11]:
#content

### 物件情報の取得

In [12]:
#物件・建物情報を変数detailに格納
detail = content.find('div',class_='cassetteitem-detail')
#変数titleに物件名を格納,text情報だけ抜くので.text
title = detail.find('div',class_='cassetteitem_content-title').text

#変数addressに住所を格納
address = detail.find('li',class_='cassetteitem_detail-col1').text

#変数accessにアクセス情報を格納
access = detail.find('li',class_='cassetteitem_detail-col2').text

#変数ageに築年数を格納
age = detail.find('li',class_='cassetteitem_detail-col3').text

In [13]:
title,address,access,age

('アイレックス',
 '東京都荒川区町屋２',
 '\n東京メトロ千代田線/町屋駅 歩7分\nＪＲ山手線/田端駅 歩24分\n京成本線/新三河島駅 歩13分\n',
 '\n築16年\n4階建\n')

### 部屋情報の取得

In [14]:
#各部屋の情報を変数tableに格納
table = content.find('div',class_='cassetteitem-item')
#table

In [15]:
#変数tableから全てのtrタグを取得、変数tr_tagsに格納
tr_tags = table.find_all('tr',class_='js-cassette_link')

#tr_tagsの中から最初の一つだけtr_tagに格納
tr_tag = tr_tags[0]
#tr_tag

In [16]:
#trの中身確認。tdの３〜６番目に情報が格納されている。
#find_allで取得した結果はリスト形式で出てくるので、カンマを使って欲しい部分を取り出す

#変数floor,price,first_fee,capacity

floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]
#floor,price,first_fee,capacity

In [17]:
# リスト形式で抽出した情報を上記は各変数に入れ込むような式の形になっている（unpackという）

In [18]:
#さらにliが入っているので分解
#変数feeとmanagement_feeに賃料と管理費を格納
fee,management_fee = price.find_all('li')

#変数depositとgratuityに敷金、礼金を格納
deposit,gratuity = first_fee.find_all('li')

#変数madoriとmensekiに間取りと面積を格納
madori,menseki = capacity.find_all('li')

fee,management_fee,deposit,gratuity,madori,menseki,madori,menseki

(<li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">14万円</span></span></li>,
 <li><span class="cassetteitem_price cassetteitem_price--administration">8000円</span></li>,
 <li><span class="cassetteitem_price cassetteitem_price--deposit">-</span></li>,
 <li><span class="cassetteitem_price cassetteitem_price--gratuity">14万円</span></li>,
 <li><span class="cassetteitem_madori">2LDK</span></li>,
 <li><span class="cassetteitem_menseki">58.78m<sup>2</sup></span></li>,
 <li><span class="cassetteitem_madori">2LDK</span></li>,
 <li><span class="cassetteitem_menseki">58.78m<sup>2</sup></span></li>)

In [19]:
### 取得した情報を辞書型に格納していく

In [20]:
#あとでデータを扱いやすくするために辞書型に格納する！

d = {
    'title':title,
    'address':address,
    'access':access,
    'age':age,
    'floor':floor.text,
    'fee':fee.text,
    'management_fee':management_fee.text,
    'deposit':deposit.text,
    'gratuity':gratuity.text,
    'madori':madori.text,
    'menseki':menseki.text
}

In [21]:
d

{'title': 'アイレックス',
 'address': '東京都荒川区町屋２',
 'access': '\n東京メトロ千代田線/町屋駅 歩7分\nＪＲ山手線/田端駅 歩24分\n京成本線/新三河島駅 歩13分\n',
 'age': '\n築16年\n4階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1階',
 'fee': '14万円',
 'management_fee': '8000円',
 'deposit': '-',
 'gratuity': '14万円',
 'madori': '2LDK',
 'menseki': '58.78m2'}

### 1ページ目から全ての情報を取得する

In [22]:
#変数d_listに空のリストを作成する
d_list = []

#全ての物件情報（２０件）を取得する
contents = soup.find_all('div',class_='cassetteitem')

#各物件情報をforループで取得する
for content in contents:
    
    #物件情報と部屋情報を取得
    detail = content.find('div',class_='cassetteitem-detail')
    table = content.find('div',class_='cassetteitem-item')
    
    #物件情報から必要な情報を取得
    title = detail.find('div',class_='cassetteitem_content-title').text
    address = detail.find('li',class_='cassetteitem_detail-col1').text
    access = detail.find('li',class_='cassetteitem_detail-col2').text
    age = detail.find('li',class_='cassetteitem_detail-col3').text
    
    #部屋情報のブロックから各部屋情報を取得
    tr_tags = table.find_all('tr',class_='js-cassette_link')
    
    #各部屋情報をforループで取得する
    for tr_tag in tr_tags:
        
        #部屋情報の行から欲しい情報を取得
        floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]
        
        #さらに細かい情報を取得
        fee,management_fee = price.find_all('li')
        deposit,gratuity = first_fee.find_all('li')
        madori,menseki = capacity.find_all('li')
        
        #取得した全ての情報を辞書に格納
        d = {
            'title':title,
            'address':address,
            'access':access,
            'age':age,
            'floor':floor.text,
            'fee':fee.text,
            'management_fee':management_fee.text,
            'deposit':deposit.text,
            'gratuity':gratuity.text,
            'madori':madori.text,
            'menseki':menseki.text
        }
        
        #取得した辞書をd_listに格納する！
        
        d_list.append(d)

In [25]:
#d_list

In [26]:
# pprintを用いると一行でずらっと記載されてしまう辞書型を開業してキーごとにわかりやすく表示しれくれる
pprint(d_list[0])
print()
pprint(d_list[1])

{'access': '\n東京メトロ千代田線/町屋駅 歩7分\nＪＲ山手線/田端駅 歩24分\n京成本線/新三河島駅 歩13分\n',
 'address': '東京都荒川区町屋２',
 'age': '\n築16年\n4階建\n',
 'deposit': '-',
 'fee': '14万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1階',
 'gratuity': '14万円',
 'madori': '2LDK',
 'management_fee': '8000円',
 'menseki': '58.78m2',
 'title': 'アイレックス'}

{'access': '\nＪＲ山手線/西日暮里駅 歩6分\n京成本線/新三河島駅 歩8分\nＪＲ京浜東北線/日暮里駅 歩14分\n',
 'address': '東京都荒川区西日暮里６',
 'age': '\n築9年\n5階建\n',
 'deposit': '4.1万円',
 'fee': '8.2万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t2階',
 'gratuity': '8.2万円',
 'madori': '1DK',
 'management_fee': '3000円',
 'menseki': '25.15m2',
 'title': 'ロイヤルシティ西日暮里'}


### 複数ページから取得する関数をつける

In [27]:
#1　アクセスするURLを設定する　→ここを２、３・・・と変更していけばよい！
#2 Requestsを使って設定したURLにアクセスする
#3 取得したHTMLをBeautifulSoupをで解析する
#4 全ての物件情報を取得する（２０件）
#5 各物件情報から物件の詳細と各部屋情報を取得する
#6　それぞれを解析する
#7　解析した結果を辞書に格納する

In [28]:
#1〜３をループする
# 3ページ目までを取得とする

for i in range(1,4):
    print('d_listの大きさ:', len(d_list))
    
    #変数target_urlにアクセス先のURLを格納する
    target_url = url.format(i)
    
    print(target_url)    
    
# 負荷をかけないようにsleepを挟み込んであげる
    sleep(1)

d_listの大きさ: 48
https://suumo.jp/chintai/tokyo/sc_arakawa/?page=1
d_listの大きさ: 48
https://suumo.jp/chintai/tokyo/sc_arakawa/?page=2
d_listの大きさ: 48
https://suumo.jp/chintai/tokyo/sc_arakawa/?page=3


In [29]:
# これまでをすべてまとめる

#1〜３をループする

for i in range(1,4):
    print('d_listの大きさ:', len(d_list)) # list の長さを出力することで正確に抽出できているかを知るため

    #変数target_urlにアクセス先のURLを格納する
    target_url = url.format(i)
    
    #target_urlへのアクセス結果を変数rに格納する→ここでデータとってきてる！
    r = requests.get(target_url)
    
    sleep(1)
    
    #取得結果を解析してsoupに格納する
    soup = BeautifulSoup(r.text)
    
    #変数d_listに空のリストを作成する
    # d_list = [] → すでに上で定義しているのでここで定義すると変になってしまう

    #全ての物件情報（２０件）を取得する
    contents = soup.find_all('div',class_='cassetteitem')

    #各物件情報をforループで取得する
    for content in contents:
    
        #物件情報と部屋情報を取得
        detail = content.find('div',class_='cassetteitem-detail')
        table = content.find('div',class_='cassetteitem-item')

        #物件情報から必要な情報を取得
        title = detail.find('div',class_='cassetteitem_content-title').text
        address = detail.find('li',class_='cassetteitem_detail-col1').text
        access = detail.find('li',class_='cassetteitem_detail-col2').text
        age = detail.find('li',class_='cassetteitem_detail-col3').text

        #部屋情報のブロックから各部屋情報を取得
        tr_tags = table.find_all('tr',class_='js-cassette_link')

        #各部屋情報をforループで取得する
        for tr_tag in tr_tags:

            #部屋情報の行から欲しい情報を取得
            floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]

            #さらに細かい情報を取得
            fee,management_fee = price.find_all('li')
            deposit,gratuity = first_fee.find_all('li')
            madori,menseki = capacity.find_all('li')

            #取得した全ての情報を辞書に格納
            d = {
                'title':title,
                'address':address,
                'access':access,
                'age':age,
                'floor':floor.text,
                'fee':fee.text,
                'management_fee':management_fee.text,
                'deposit':deposit.text,
                'gratuity':gratuity.text,
                'madori':madori.text,
                'menseki':menseki.text
            }

            #取得した辞書をd_listに格納する！

            d_list.append(d)

d_listの大きさ: 48
d_listの大きさ: 96
d_listの大きさ: 149


In [30]:
#取得したデータ結果を表形式に変える！
#headで先頭業を確認

df = pd.DataFrame(d_list)
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,アイレックス,東京都荒川区町屋２,\n東京メトロ千代田線/町屋駅 歩7分\nＪＲ山手線/田端駅 歩24分\n京成本線/新三河島...,\n築16年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,14万円,8000円,-,14万円,2LDK,58.78m2
1,ロイヤルシティ西日暮里,東京都荒川区西日暮里６,\nＪＲ山手線/西日暮里駅 歩6分\n京成本線/新三河島駅 歩8分\nＪＲ京浜東北線/日暮里...,\n築9年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,8.2万円,3000円,4.1万円,8.2万円,1DK,25.15m2
2,ロイヤルシティ西日暮里,東京都荒川区西日暮里６,\nＪＲ山手線/西日暮里駅 歩6分\n京成本線/新三河島駅 歩8分\nＪＲ京浜東北線/日暮里...,\n築9年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,8.2万円,5000円,4.1万円,8.2万円,1DK,25.15m2
3,ＪＲ山手線 西日暮里駅 5階建 築9年,東京都荒川区西日暮里６,\nＪＲ山手線/西日暮里駅 歩6分\nＪＲ常磐線/三河島駅 歩10分\nＪＲ京浜東北線/日暮...,\n築9年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,8.2万円,5000円,4.1万円,8.2万円,1DK,25.15m2
4,東京メトロ千代田線 町屋駅 3階建 築8年,東京都荒川区東尾久６,\n東京メトロ千代田線/町屋駅 歩13分\n日暮里・舎人ライナー/熊野前駅 歩8分\nＪＲ山...,\n築8年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,7.5万円,3000円,7.5万円,-,1K,26.7m2


In [31]:
#uniqueで重複なく取り出せる→２０件×3ページになっているか確認
len(df.title.unique())

60

In [32]:

#encordinghはエクセル文字化け対策！
df.to_csv('chintai_arakata.csv',index=None,encoding='utf-8-sig')

In [33]:
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,アイレックス,東京都荒川区町屋２,\n東京メトロ千代田線/町屋駅 歩7分\nＪＲ山手線/田端駅 歩24分\n京成本線/新三河島...,\n築16年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,14万円,8000円,-,14万円,2LDK,58.78m2
1,ロイヤルシティ西日暮里,東京都荒川区西日暮里６,\nＪＲ山手線/西日暮里駅 歩6分\n京成本線/新三河島駅 歩8分\nＪＲ京浜東北線/日暮里...,\n築9年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,8.2万円,3000円,4.1万円,8.2万円,1DK,25.15m2
2,ロイヤルシティ西日暮里,東京都荒川区西日暮里６,\nＪＲ山手線/西日暮里駅 歩6分\n京成本線/新三河島駅 歩8分\nＪＲ京浜東北線/日暮里...,\n築9年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,8.2万円,5000円,4.1万円,8.2万円,1DK,25.15m2
3,ＪＲ山手線 西日暮里駅 5階建 築9年,東京都荒川区西日暮里６,\nＪＲ山手線/西日暮里駅 歩6分\nＪＲ常磐線/三河島駅 歩10分\nＪＲ京浜東北線/日暮...,\n築9年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,8.2万円,5000円,4.1万円,8.2万円,1DK,25.15m2
4,東京メトロ千代田線 町屋駅 3階建 築8年,東京都荒川区東尾久６,\n東京メトロ千代田線/町屋駅 歩13分\n日暮里・舎人ライナー/熊野前駅 歩8分\nＪＲ山...,\n築8年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,7.5万円,3000円,7.5万円,-,1K,26.7m2


In [34]:
df["floor"].head()

0    \r\n\t\t\t\t\t\t\t\t\t\t\t1階
1    \r\n\t\t\t\t\t\t\t\t\t\t\t2階
2    \r\n\t\t\t\t\t\t\t\t\t\t\t2階
3    \r\n\t\t\t\t\t\t\t\t\t\t\t2階
4    \r\n\t\t\t\t\t\t\t\t\t\t\t1階
Name: floor, dtype: object

In [38]:
df.dtypes

title             object
address           object
access            object
age               object
floor             object
fee               object
management_fee    object
deposit           object
gratuity          object
madori            object
menseki           object
dtype: object

In [48]:
# floorの余計な文字を削除
df["floor"].str.replace("\r\n\t\t\t\t\t\t\t\t\t\t\t","")

0      1階
1      2階
2      2階
3      2階
4      1階
       ..
189    1階
190    2階
191    6階
192    1階
193    2階
Name: floor, Length: 194, dtype: object

In [51]:
# 以下、間違って消してしまったが、feeの万円を削除
# df["fee"].str.replace("万円","")

In [53]:
df['fee'] = df['fee'].astype("float")

In [56]:
df["fee"] = df["fee"]*10000

In [58]:
df['fee'] = df['fee'].astype("int")

In [60]:
# df

### FeeとFloorについて整形したので、それをスプシに一旦落として提出

In [61]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [62]:
#jsonファイルを使って認証情報を取得
SCOPES = [    'https://www.googleapis.com/auth/spreadsheets',    'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = "my-project-scraping-389714-3721aa5b8863.json"

In [63]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)

In [64]:
#認証情報を使ってスプレッドシートの操作権を取得
gs = gspread.authorize(credentials)

In [65]:
#共有したスプレッドシートのキー（後述）を使ってシートの情報を取得
# スプレッドシートのキーはシートのURLのうち、最後のeditのteamが書いてある一つ前のセクションから抜き出し
SPREADSHEET_KEY = "1mA-Zv-h7OvzEFmzCYoS6btqzxW9oGPZPLqSCG9ynVYE"

In [66]:
from gspread_dataframe import set_with_dataframe

In [67]:
workbook = gs.open_by_key(SPREADSHEET_KEY)

In [69]:
#スプしにワークシートを追加（rows,colsは必須！）
workbook.add_worksheet(title='賃貸情報',rows=300 ,cols=15)

<Worksheet '賃貸情報' id:851647671>

In [ ]:
# スプシにデータを出力